<a href="https://colab.research.google.com/github/safdarjung/DL_Projects/blob/main/Pytorch_MultiClassification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

In [ ]:
digits = load_digits()
X, y = digits.data, digits.target

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

In [ ]:
class CustomDataset(Dataset):
  def __init__(self, data, target):
    self.data = torch.tensor(data, dtype=torch.float32)
    self.target = torch.tensor(target, dtype=torch.long)

  def __len__(self):
    return len(self.data)

  def __getitem__(self, idx):
    sample = {"data": self.data[idx], "target":self.target[idx]}
    return sample

In [ ]:
train_dataset = CustomDataset(x_train, y_train)
test_dataset = CustomDataset(x_test, y_test)


In [ ]:
len(train_dataset)

1437

In [ ]:
len(test_dataset)

360

In [ ]:
train_data_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
test_data_loader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=2)


In [ ]:
class SimpleNN(nn.Module):
  def __init__(self, input_size, hidden_size, num_classes):
    super(SimpleNN, self).__init__()
    self.layer1 = nn.Linear(input_size, hidden_size)
    self.relu = nn.ReLU()
    self.layer2 = nn.Linear(hidden_size, num_classes)

  def forward(self,x):
    out = self.layer1(x)
    out = self.relu(out)
    out = self.layer2(out)
    return out

In [ ]:
input_size = x_train.shape[1]
hidden_size = 64
output_size = len(set(y_train))


In [ ]:
model = SimpleNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [ ]:

num_epochs = 100

for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0
  for batch in train_data_loader:
    data = batch["data"]
    target = batch["target"]

    optimizer.zero_grad()
    output = model(data)

    loss = criterion(output, target)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()

  epoch_loss = running_loss / len(train_data_loader)
  print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}")

Epoch [1/100], Loss: 1.9051
Epoch [2/100], Loss: 1.1041
Epoch [3/100], Loss: 0.5971
Epoch [4/100], Loss: 0.3713
Epoch [5/100], Loss: 0.2627
Epoch [6/100], Loss: 0.2006
Epoch [7/100], Loss: 0.1620
Epoch [8/100], Loss: 0.1341
Epoch [9/100], Loss: 0.1139
Epoch [10/100], Loss: 0.0981
Epoch [11/100], Loss: 0.0853
Epoch [12/100], Loss: 0.0747
Epoch [13/100], Loss: 0.0656
Epoch [14/100], Loss: 0.0578
Epoch [15/100], Loss: 0.0514
Epoch [16/100], Loss: 0.0462
Epoch [17/100], Loss: 0.0414
Epoch [18/100], Loss: 0.0375
Epoch [19/100], Loss: 0.0338
Epoch [20/100], Loss: 0.0304
Epoch [21/100], Loss: 0.0279
Epoch [22/100], Loss: 0.0254
Epoch [23/100], Loss: 0.0233
Epoch [24/100], Loss: 0.0214
Epoch [25/100], Loss: 0.0195
Epoch [26/100], Loss: 0.0179
Epoch [27/100], Loss: 0.0166
Epoch [28/100], Loss: 0.0153
Epoch [29/100], Loss: 0.0142
Epoch [30/100], Loss: 0.0131
Epoch [31/100], Loss: 0.0122
Epoch [32/100], Loss: 0.0114
Epoch [33/100], Loss: 0.0106
Epoch [34/100], Loss: 0.0099
Epoch [35/100], Loss: 0

In [ ]:
model.eval()
all_pred = []
all_targets = []

with torch.no_grad():
  for batch in test_data_loader:
    data = batch["data"]
    target = batch["target"]
    output = model(data)
    pred = torch.argmax(output, 1)
    all_pred.extend(pred.numpy())
    all_targets.extend(target.numpy())

accuracy = accuracy_score(all_targets, all_pred)

print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 97.22%
